## Image Alteration Detection

#### Import libraries

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
print("hello")

hello


#### Defining required parameters

In [8]:
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 5  # increase this for better results

# Paths
train_dir = 'dataset/train'
test_dir = 'dataset/test_mixed'

#### Generating data

In [9]:
# Data Generator
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               validation_split=0.2)

train_data = train_gen.flow_from_directory(train_dir,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode='binary',
                                           subset='training')

val_data = train_gen.flow_from_directory(train_dir,
                                         target_size=IMG_SIZE,
                                         batch_size=BATCH_SIZE,
                                         class_mode='binary',
                                         subset='validation')

Found 108 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


#### Creating model

In [10]:
# Build Model with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden

#### Training model

In [ ]:
# Train
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

#### Predicting model

In [ ]:
# Predict on mixed test folder
def predict_mixed_folder(test_dir):
    predictions = []
    files = os.listdir(test_dir)
    for fname in files:
        img_path = os.path.join(test_dir, fname)
        img = load_img(img_path, target_size=IMG_SIZE)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        prob = model.predict(img_array)[0][0]
        label = 'Fake' if prob > 0.5 else 'Real'
        predictions.append((fname, label, prob))

    return predictions

#### Showing the result

In [ ]:
# Run and show results
print("Hello")
results = predict_mixed_folder(test_dir)
for fname, label, prob in results:
    print(f"{fname}: {label} ({prob:.2f})")